# image2text_note
pyocrライブラリを介して、tesseractを利用して文字認識を行います。  

> Copyright [zashio]  
>   
> Licensed under the Apache License, Version 2.0 (the "License");  
> you may not use this file except in compliance with the License.  
> You may obtain a copy of the License at  
 
>     http://www.apache.org/licenses/LICENSE-2.0  
  
> Unless required by applicable law or agreed to in writing, software  
> distributed under the License is distributed on an "AS IS" BASIS,  
> WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.  
> See the License for the specific language governing permissions and  
> limitations under the License.  

### importするライブラリの紹介
pillow( PIL ) : OpenCVのような高度な処理はできないが、OpenCVよりもコードが平易  
pyocr : pythonでtesseractなどのOCRライブラリを利用するためのライブラリ  
tesseract : 現在はGoogleが開発しているオープンソースのOCRライブラリ(PCにインストール済み)

In [ ]:
from PIL import Image, ImageDraw, ImageFont
import sys

import pyocr
import pyocr.builders

### 初期設定
tool : tesseract使うため、0番目の要素を指定  
use_rang : Japaneseを使うため、1番目の要素を指定  
filepath : 画像ファイルへの相対パス  

In [ ]:
tool = pyocr.get_available_tools()[0]
use_lang = tool.get_available_languages()[1]
filepath = 'resource/Hanada-Ritsumen800.png'

### 画像の前処理
グレイスケールで読み込み、2値化  
再びRGB画像に変換(後ほど青色や赤色で描画するため、3チャンネル画像でなければならない)  

In [ ]:
image = Image.open(filepath).convert("L")
image = image.point(lambda x: 0 if x < 128 else x) 


image = Image.merge(
    "RGB",
    (   
        image.point(lambda x: x),
        image.point(lambda x: x),
        image.point(lambda x: x)
    )
)


### 画像からテキストを取得

In [ ]:
txt = tool.image_to_string(
    image,
    lang=use_lang,
    builder=pyocr.builders.TextBuilder()
)

### 文字の位置(box)も取得
今回はこれを利用  
word_boxesは文字列とその文字列を囲む長方形の座標の情報を保持

In [ ]:
word_boxes = tool.image_to_string(
    image,
    lang=use_lang,
    builder=pyocr.builders.WordBoxBuilder()
)

### 行ごとまとめて取得

In [ ]:
line_and_word_boxes = tool.image_to_string(
    image,
    lang=use_lang,
    builder=pyocr.builders.LineBoxBuilder()
)

### boxを描画する関数を定義

In [ ]:
def draw_boxes(image, word_boxes, color):
    """長方形の左上と右下の頂点を受け取り画像に描画する"""
    draw = ImageDraw.Draw(image)

    for word_box in word_boxes:
        
        pos1_x = word_box.position[0][0]
        pos1_y = word_box.position[0][1]
        pos2_x = word_box.position[1][0]
        pos2_y = word_box.position[1][1]
        
        draw.polygon([
            pos1_x, pos1_y,
            pos2_x, pos1_y,
            pos2_x, pos2_y,
            pos1_x, pos2_y], None, color)
            
    return image

### Textを描画する関数を定義

In [ ]:
def draw_texts(image, word_boxes, color):
    draw = ImageDraw.Draw(image)
    fnt = ImageFont.truetype('resource/Kokoro.otf',65)

#    日本語文を扱う場合には、上記のようにフォントを設定する必要がある
#    draw.text()の引数に、font=fntを追加する。
    
    for word_box in word_boxes:
        
        pos1_x = word_box.position[0][0]
        pos2_y = word_box.position[1][1]
    
        draw_position = (pos1_x, pos2_y)
        
        content = word_box.content
        
        draw.text(draw_position, content, color, font=fnt)
        
    return image

### 上記の2つの描画関数を実行

In [ ]:
draw_boxes(image, word_boxes, 'Red')
draw_texts(image, word_boxes, 'blue')

In [ ]:
image.save('resource/OutHanada-Ritsumen800.png')